In [ ]:
### implement cosine similarity
### <x, y> / ||x|| ||y||
import numpy as np

def cosSim(x,y):
    return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    
def pearsonCorr(x,y):
    a = x-np.mean(x)
    b = y-np.mean(y)
    if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
        return 0
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))
    
    
### implement nearestNeighbor, compute all similarities between users, rank them and return the ranked list
### input is: prefs, personName, distance function
## output a ranked list
## prefs = {1:{'python':1, 'java':2, 'c':3}, 2: {'python':3, 'scala':2, 'java':1}, 3: {'python':2, 'c++': 1, 'java': 3}}


scores = []
for name in prefs:
    if (person != name):
        scores.append((distance_func(prefs[person],prefs[name]),name))

scores.sort(reverse = True)

return scores

### input prefs, personName, distance function
### output recommended movies in a ranked list
sum (s_i * r_i) / sum (s_i)

def recommend(prefs, person, distance_func):
    weighted_totals = {}
    weights = {}
    threshold = 0.1
    for p in prefs:
        if (p != person):
            sim = distance_func(prefs[person],prefs[p])
            for item in pref[p]:
                if (item not in pref[person]):
                    if (item not in weighted_totals):
                        weighted_totals.setdefault(item, 0)
                        weights.setdefault(item,0)
                     weighted_totals[item] += sim * pref[p][item]
                     weights[item] += sim
     result_list = [(weighted_totals[i]/weights[i], i) for i in weights]
     return result_list.sort(reverse=True)     



In [38]:
import math
import pandas as pd
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [30]:
ratings.movieId.max()

193609

In [108]:
def ratings2dict(ratings):
    prefs = {}
    for user_id in ratings.userId.unique():
        prefs[user_id] = {}
    for row in ratings.values:
        prefs[int(row[0])][int(row[1])] = row[2]
    return prefs

def cosine_dic(dic1,dic2):
    numerator = 0
    denx = 0
    for key1,val1 in dic1.items():
        numerator += val1*dic2.get(key1,0.0)
        denx += val1*val1
    deny = 0
    for val2 in dic2.values():
        deny += val2*val2
    return numerator/math.sqrt(denx*deny)

# def cosSim(x,y):
#     if np.linalg.norm(x) == 0 or np.linalg.norm(y) == 0:
#         return 0
#     return np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))

def pearson_dic(dic1,dic2):
    x_bar = mean(dic1.values())
    y_bar = mean(dic2.values())
    numerator = 0
    denx = 0
    for key1,val1 in dic1.items():
        numerator += (val1-x_bar)*(dic2.get(key1,0.0)-y_bar)
        denx += (val1-x_bar)*(val1-x_bar)
    deny = 0
    for val2 in dic2.values():
        deny += (val2-y_bar)*(val2-y_bar)
    return numerator/math.sqrt(denx*deny)
    
        
# def pearsonCorr(x,y):
#     a = x-np.mean(x)
#     b = y-np.mean(y)
#     if np.linalg.norm(a) == 0 or np.linalg.norm(b) == 0:
#         return 0
#     return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))


def find_similar_users(prefs,userId,distance_func):
    scores = []   
    for user_id in prefs:
        if (userId != user_id):
            scores.append((distance_func(prefs[user_id],prefs[userId]),userId))
    scores.sort(reverse = True)
    return scores

def recommend(prefs, userId, distance_func,topk=10):
    weighted_totals = {}
    weights = {}
    for p in prefs:
        if (p != userId):
            sim = distance_func(prefs[userId],prefs[p])
            for item in prefs[p]:
                if (item not in prefs[userId]):
                    weighted_totals[item] = weighted_totals.get(item,0)+sim * prefs[p][item]
                    weights[item] = weights.get(item,0)+sim
                    
#                     if (item not in weighted_totals):
#                         weighted_totals.setdefault(item, 0)
#                         weights.setdefault(item,0)
#                     weighted_totals[item] += sim * prefs[p][item]
#                     weights[item] += sim
    result_list = [(weighted_totals[i]/weights[i], i) for i in weights]
    result_list.sort(reverse=True)    
    return result_list[:topk]

In [112]:
prefs = ratings2dict(ratings)
user1 = recommend(prefs,1,cosine_dic)
user1

/Users/carold/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:71: RuntimeWarning: invalid value encountered in double_scalars


[(5.000000000000001, 176601),
 (5.000000000000001, 6835),
 (5.000000000000001, 5746),
 (5.0, 179133),
 (5.0, 175431),
 (5.0, 175397),
 (5.0, 175387),
 (5.0, 175293),
 (5.0, 174551),
 (5.0, 173963)]

In [110]:
ratings[ratings.movieId == 176601]

,userId,movieId,rating,timestamp
9147,62,176601,5.0,1525795252


In [37]:
ratings_matrix = ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix.fillna(0, inplace = True )
ratings_matrix.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
ratings_matrix.shape

(9724, 610)

In [11]:
#Calculating Cosine Similiarities
movie_similarity = 1 - pairwise_distances(ratings_matrix.values, metric="cosine" )
np.fill_diagonal(movie_similarity, 0) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix = pd.DataFrame( movie_similarity )
ratings_matrix.head(5)

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,0.000000,0.924008,0.807673,0.309547,0.783011,0.888195,0.751479,0.569005,0.440312,0.911234,...,0.035955,0.035955,0.035955,0.035955,0.035955,0.035955,0.035955,0.035955,0.035955,0.233049
1,0.924008,0.000000,0.797331,0.336409,0.781272,0.849916,0.740089,0.597975,0.411897,0.900694,...,0.040693,0.040693,0.040693,0.040693,0.040693,0.040693,0.040693,0.040693,0.040693,0.231666
2,0.807673,0.797331,0.000000,0.393742,0.807746,0.744898,0.778451,0.683278,0.523208,0.838544,...,0.016776,0.016776,0.016776,0.016776,0.016776,0.016776,0.016776,0.016776,0.016776,0.123567
3,0.309547,0.336409,0.393742,0.000000,0.501289,0.337697,0.499569,0.371673,0.265316,0.352659,...,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.000317,0.020985
4,0.783011,0.781272,0.807746,0.501289,0.000000,0.676944,0.773350,0.599335,0.423186,0.759508,...,0.023469,0.023469,0.023469,0.023469,0.023469,0.023469,0.023469,0.023469,0.023469,0.148257


In [12]:
ratings_matrix.shape

(9724, 9724)